In [1]:
import pandas as pd
import requests

In [2]:
df_trans = pd.read_csv('../data/수정 후 데이터/연평균 지하철 이용객.csv')
df_trans.head()

,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,07시-08시 하차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,1시이후 승차,1시이후 하차,총 이용객,유임승차인원,무임승차인원,유임하차인원,무임하차인원
0,1호선,가능,378.566667,6.050000,7903.966667,1626.133333,12657.900000,4374.083333,24975.150000,5366.183333,...,6906.650000,112.983333,1916.666667,0.050000,1.650000,5.938758e+05,146352.583333,55133.616667,136380.10,54102.966667
1,1호선,가산디지털단지,1340.000000,6.133333,8205.400000,1980.583333,7914.766667,10941.750000,14763.350000,46802.533333,...,6686.566667,182.500000,1090.016667,0.700000,2.033333,1.180312e+06,418268.766667,29773.750000,497380.25,32982.716667
2,1호선,간석,375.650000,10.866667,5440.366667,1158.600000,11762.750000,2589.983333,26198.050000,4138.650000,...,7005.550000,146.683333,2437.116667,0.116667,2.066667,5.545781e+05,139479.600000,41719.150000,131105.05,40367.766667
3,1호선,개봉,1226.050000,8.633333,18414.966667,5494.800000,36569.333333,11855.966667,95251.200000,16530.883333,...,24220.900000,175.616667,3167.933333,0.300000,2.283333,1.524459e+06,514742.366667,145080.383333,511793.65,150936.250000
4,1호선,관악,187.150000,1.800000,6983.550000,1931.716667,13008.033333,4626.450000,34716.916667,6062.566667,...,8734.116667,52.500000,980.500000,0.066667,0.266667,6.517340e+05,182214.900000,50254.650000,168526.15,48831.800000


In [3]:
# kakao key 불러오기
with open('C:/workspace/api_key/kakao_key.txt') as file:
    kakao_key = file.read()

In [4]:
# kakao_key를 통해서 모든 지하철역 주소 위도 경도 좌표 구하기
url = "https://dapi.kakao.com/v2/local/search/keyword.json?"
header = {"Authorization": "KakaoAK " + kakao_key}
addr, lngs, lats = [], [], []
for i in range(len(df_trans)):
    query = df_trans['지하철역'][i]
    if '역' not in df_trans['지하철역'][i]:
        station = df_trans['지하철역'][i]
        if '(' in df_trans['지하철역'][i]:
            station = df_trans['지하철역'][i].split('(')[0]
        query = station + '역'
    result = requests.get(url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + kakao_key}).json()['documents']
    try:
        addr.append(result[0]['address_name'])
        lngs.append(float(result[0]['x']))
        lats.append(float(result[0]['y']))
    except:
        print(df_trans['지하철역'][i])

In [5]:
df_trans['주소'] = pd.DataFrame(addr)
df_trans['위도'] = pd.DataFrame(lats)
df_trans['경도'] = pd.DataFrame(lngs)

In [6]:
df_trans_seoul = df_trans[df_trans['주소'].str.contains('서울')]

In [7]:
df_trans_seoul = df_trans_seoul.set_index('지하철역').reset_index()

In [8]:
df_trans_seoul.to_csv('../data/수정 후 데이터/seoul_stations.csv', index=False)